In [1]:
#from _future_ import print_function

from pyspark.sql import SparkSession
# $example on:schema_merging$
from pyspark.sql import Row
# $example off:schema_merging$

from pyspark.sql.window import Window

from pyspark.sql.functions import dense_rank

# using SQLContext to read parquet file
from pyspark.sql import SQLContext

In [2]:
path = "D://_E/Spark_training/SparkPythonTraining/Task5/"  

In [9]:
spark = SparkSession.builder \
    .master('local') \
    .appName('Python Spark SQL csv vs parket example') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

In [10]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [12]:
%%time
#csv
df_s = spark.read.load(path + "stuff.csv",
                         format="csv", sep=",", inferSchema="true", header="true")
#df_oi.show()

Wall time: 3.03 s


In [11]:
%%time
# to read parquet file
df_sp = sqlContext.read.parquet(path + "stuff.parquet")
#df_sp.show()

Wall time: 973 ms


In [22]:
df_s.registerTempTable("stuff")
df_sp.registerTempTable("stuff_parquet")
top_n = '5'

In [23]:
%%time
stuff_top_n = sqlContext.sql("SELECT SUM(duration), from FROM stuff GROUP BY 2 ORDER BY SUM(duration) DESC LIMIT " + top_n)
stuff_top_n.show()

+-------------+--------+
|sum(duration)|    from|
+-------------+--------+
|      7609893|   Gomel|
|      7606500|   Minsk|
|      7600506|Warshava|
|      7592524|  Berlin|
|      7585043| Vitebsk|
+-------------+--------+

Wall time: 3.49 s


In [24]:
%%time
stuff_top_n = sqlContext.sql("SELECT SUM(duration), from FROM stuff_parquet GROUP BY 2 ORDER BY SUM(duration) DESC LIMIT " + top_n)
stuff_top_n.show()

+-------------+-------+
|sum(duration)|   from|
+-------------+-------+
|      7612205| Berlin|
|      7607177|Vinitsa|
|      7601572|  Paris|
|      7599302|  Brest|
|      7598566|  Gomel|
+-------------+-------+

Wall time: 1.85 s


In [25]:
spark.stop()